In [0]:
# Importing Libraries

In [0]:
import pandas as pd
import numpy as np
import os

In [0]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [0]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [0]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [0]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'/content/drive/My Drive/HumanActivityRecognition.zip (Unzipped Files)/HAR/UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [0]:
def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'/content/drive/My Drive/HumanActivityRecognition.zip (Unzipped Files)/HAR/UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).as_matrix()

In [0]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [9]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

In [0]:
# Configuring a session
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [11]:
# Import Keras
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

Using TensorFlow backend.


In [0]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

In [0]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [0]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [16]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()


In [17]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


- Defining the Architecture of LSTM

In [0]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [0]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 7352 samples, validate on 2947 samples
Epoch 1/30




7352/7352 [==============================] - 92s 12ms/step - loss: 1.3516 - acc: 0.4373 - val_loss: 1.1546 - val_acc: 0.4968
Epoch 2/30
7352/7352 [==============================] - 83s 11ms/step - loss: 0.9905 - acc: 0.5813 - val_loss: 0.9254 - val_acc: 0.5789
Epoch 3/30
7352/7352 [==============================] - 82s 11ms/step - loss: 0.7942 - acc: 0.6484 - val_loss: 0.8159 - val_acc: 0.6138
Epoch 4/30
7352/7352 [==============================] - 82s 11ms/step - loss: 0.7300 - acc: 0.6591 - val_loss: 0.7393 - val_acc: 0.6162
Epoch 5/30
7352/7352 [==============================] - 84s 11ms/step - loss: 0.6787 - acc: 0.6634 - val_loss: 0.7693 - val_acc: 0.6223
Epoch 6/30
7352/7352 [==============================] - 86s 12ms/step - loss: 0.6435 - acc: 0.6684 - val_loss: 0.7687 - val_acc: 0.6026
Epoch 7/30
7352/7352 [==============

In [0]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  ...  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS
True                                 ...                                      
LAYING                 510        0  ...                   0                 0
SITTING                  0      407  ...                   0                 0
STANDING                 0      106  ...                   0                 0
WALKING                  0        1  ...                  10                14
WALKING_DOWNSTAIRS       0        0  ...                 384                11
WALKING_UPSTAIRS         0        6  ...                  13               437

[6 rows x 6 columns]


In [0]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 6s 2ms/step


In [0]:
score

[0.38706818539458054, 0.8941296233457754]

- With a simple 2 layer architecture we got 90.09% accuracy and a loss of 0.30
- We can further imporve the performace with Hyperparameter tuning

In [0]:
lu=[8,16,64,128,256]
for n_hidden in lu:
  print(n_hidden)
  model = Sequential()
  model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
  model.add(Dropout(0.5))
  model.add(Dense(n_classes, activation='sigmoid'))
  model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
  model.fit(X_train, Y_train, batch_size=batch_size, validation_data=(X_test, Y_test), epochs=epochs)

8
Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 89s 12ms/step - loss: 1.6115 - acc: 0.3658 - val_loss: 1.4700 - val_acc: 0.4751
Epoch 2/30
7352/7352 [==============================] - 87s 12ms/step - loss: 1.4646 - acc: 0.3817 - val_loss: 1.3813 - val_acc: 0.3811
Epoch 3/30
7352/7352 [==============================] - 87s 12ms/step - loss: 1.3209 - acc: 0.4368 - val_loss: 1.2607 - val_acc: 0.4333
Epoch 4/30
7352/7352 [==============================] - 86s 12ms/step - loss: 1.2147 - acc: 0.4718 - val_loss: 1.1651 - val_acc: 0.4873
Epoch 5/30
7352/7352 [==============================] - 87s 12ms/step - loss: 1.1292 - acc: 0.4815 - val_loss: 1.0656 - val_acc: 0.4893
Epoch 6/30
7352/7352 [==============================] - 87s 12ms/step - loss: 1.0493 - acc: 0.5054 - val_loss: 0.9974 - val_acc: 0.4815
Epoch 7/30
7352/7352 [==============================] - 87s 12ms/step - loss: 1.0302 - acc: 0.5137 - val_loss: 0.9866 - val_acc: 0.485

256 lstm units give the most accuracy

In [0]:
dr=[0.4,0.5,0.6]
for drr in dr:
  print(drr)
  model = Sequential()
  model.add(LSTM(256, input_shape=(timesteps, input_dim)))
  model.add(Dropout(drr))
  model.add(Dense(n_classes, activation='sigmoid'))
  model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
  model.fit(X_train, Y_train, batch_size=batch_size, validation_data=(X_test, Y_test), epochs=epochs)

0.4
Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 90s 12ms/step - loss: 1.3455 - acc: 0.4166 - val_loss: 1.7455 - val_acc: 0.3994
Epoch 2/30
7352/7352 [==============================] - 88s 12ms/step - loss: 1.0051 - acc: 0.5597 - val_loss: 0.7928 - val_acc: 0.6312
Epoch 3/30
7352/7352 [==============================] - 88s 12ms/step - loss: 0.8160 - acc: 0.6674 - val_loss: 0.9453 - val_acc: 0.6647
Epoch 4/30
7352/7352 [==============================] - 87s 12ms/step - loss: 0.5174 - acc: 0.8014 - val_loss: 0.8052 - val_acc: 0.5993
Epoch 5/30
7352/7352 [==============================] - 88s 12ms/step - loss: 0.3821 - acc: 0.8679 - val_loss: 0.3723 - val_acc: 0.8744
Epoch 6/30
7352/7352 [==============================] - 87s 12ms/step - loss: 0.2345 - acc: 0.9170 - val_loss: 0.2501 - val_acc: 0.9046
Epoch 7/30
7352/7352 [==============================] - 87s 12ms/step - loss: 0.2180 - acc: 0.9195 - val_loss: 0.2681 - val_acc: 0.9

In [0]:
dr=[0.7,0.8]
for drr in dr:
  print(drr)
  model = Sequential()
  model.add(LSTM(256, input_shape=(timesteps, input_dim)))
  model.add(Dropout(drr))
  model.add(Dense(n_classes, activation='sigmoid'))
  model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
  model.fit(X_train, Y_train, batch_size=batch_size, validation_data=(X_test, Y_test), epochs=epochs)

0.7



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 7352 samples, validate on 2947 samples
Epoch 1/30




7352/7352 [==============================] - 128s 17ms/step - loss: 1.2932 - acc: 0.4331 - val_loss: 1.4043 - val_acc: 0.4445
Epoch 2/30
7352/7352 [==============================] - 120s 16ms/step - loss: 1.0410 - acc: 0.5491 - val_loss: 0.8182 - val_acc: 0.5765
Epoch 3/30
7352/7352 [==============================] - 120s 16ms/step - loss: 0.8322 - acc: 0.6099 - val_loss: 0.7209 - val_acc: 0.6084
Epoch 4/30
7352/7352 [==============================] - 119s 16ms/step - loss: 0.6272 - acc: 0.7476 - val_loss: 0.4798 - val_acc: 0.8324
Epoch 5/30
7352/7352 [==============================] - 119s 16ms/step - loss: 0.4001 - acc: 0.8682 - val_loss: 0.5695 - val_acc: 0.8487
Epoch 6/30
7352/7352 [=====================

0.4 dropout rate gives highest accuracy

In [0]:
model = Sequential()
model.add(LSTM(64, input_shape=(timesteps, input_dim),return_sequences=True))
model.add(Dropout(0.65))
model.add(LSTM(32))
model.add(Dropout(0.65))
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 128, 64)           18944     
_________________________________________________________________
dropout_3 (Dropout)          (None, 128, 64)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dropout_4 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 198       
Total params: 31,558
Trainable params: 31,558
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(X_train, Y_train, batch_size=batch_size, validation_data=(X_test, Y_test), epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 235s 32ms/step - loss: 1.2478 - acc: 0.4959 - val_loss: 1.0004 - val_acc: 0.5972
Epoch 2/30
7352/7352 [==============================] - 232s 32ms/step - loss: 0.9087 - acc: 0.6299 - val_loss: 0.7938 - val_acc: 0.6837
Epoch 3/30
7352/7352 [==============================] - 231s 31ms/step - loss: 0.7754 - acc: 0.7031 - val_loss: 0.7901 - val_acc: 0.6373
Epoch 4/30
7352/7352 [==============================] - 233s 32ms/step - loss: 0.6622 - acc: 0.7394 - val_loss: 0.6739 - val_acc: 0.6936
Epoch 5/30
7352/7352 [==============================] - 232s 32ms/step - loss: 0.5545 - acc: 0.7764 - val_loss: 0.8614 - val_acc: 0.7011
Epoch 6/30
7352/7352 [==============================] - 231s 31ms/step - loss: 0.5255 - acc: 0.7847 - val_loss: 0.4699 - val_acc: 0.7872
Epoch 7/30
7352/7352 [==============================] - 234s 32ms/step - loss: 0.4575 - acc: 0.8245 - val_loss: 0.4801 - val_acc: 

Of all our models, single layer LSTM with 256 units and dropout rate of 0.4 gives us the highest accuracy

In [18]:
from keras.callbacks import ModelCheckpoint
model = Sequential()
model.add(LSTM(256, input_shape=(timesteps, input_dim)))
model.add(Dropout(0.4))
model.add(Dense(n_classes, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
filepath='weights_copy.best.hdf5'
checkpoint_1 = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
cbl=[checkpoint_1]
model.fit(X_train, Y_train, batch_size=batch_size, validation_data=(X_test, Y_test), epochs=epochs,callbacks=cbl)




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 7352 samples, validate on 2947 samples
Epoch 1/30




7352/7352 [==============================] - 87s 12ms/step - loss: 1.4140 - acc: 0.3834 - val_loss: 1.4226 - val_acc: 0.4174

Epoch 00001: val_acc improved from -inf to 0.41737, saving model to weights_copy.best.hdf5
Epoch 2/30
7352/7352 [==============================] - 79s 11ms/step - loss: 1.1025 - acc: 0.5101 - val_loss: 0.7636 - val_acc: 0.6512

Epoch 00002: val_acc improved from 0.41737 to 0.65117, saving model to weights_copy.best.hdf5
Epoch 3/30
7352/7352 [==============================] - 80s 11ms/step - loss: 0.6284 - acc: 0.7348 - val_loss: 0.6704 - val_acc: 0.7251

Epoch 00003: val_acc improved from 0.65117 to 0.72514, saving model to weights_copy.best.hdf5
Epoch 4/30
7352/7352 [====================

In [0]:
model.load_weights('weights_copy.best.hdf5')

In [20]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  ...  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS
True                                 ...                                      
LAYING                 537        0  ...                   0                 0
SITTING                  0      401  ...                   0                 2
STANDING                 0       54  ...                   0                 0
WALKING                  0        1  ...                  26                 1
WALKING_DOWNSTAIRS       0        0  ...                 420                 0
WALKING_UPSTAIRS         0        1  ...                   9               459

[6 rows x 6 columns]


In [21]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 5s 2ms/step


In [22]:
score

[0.37891137746417447, 0.9368849677638276]

### Conclusion

In [38]:
from prettytable import PrettyTable
pt=PrettyTable()
pt.field_names=['model','train_acc','test_acc']
pt.add_row(['model 1(1 LSTM layer with tuned LSTM units)','0.9446','0.8941'])
pt.add_row(['model 2(1 LSTM layer with tuned LSTM units and Dropout Rate)','0.9476','0.936885'])
pt.add_row(['model 3(2 LSTM Layers with higher dropout rate)','0.9342','0.922'])

print(pt)

+--------------------------------------------------------------+-----------+----------+
|                            model                             | train_acc | test_acc |
+--------------------------------------------------------------+-----------+----------+
|         model 1(1 LSTM layer with tuned LSTM units)          |   0.9446  |  0.8941  |
| model 2(1 LSTM layer with tuned LSTM units and Dropout Rate) |   0.9476  | 0.936885 |
|       model 3(2 LSTM Layers with higher dropout rate)        |   0.9342  |  0.922   |
+--------------------------------------------------------------+-----------+----------+


With single LSTM layer with 256 LSTM units and dropout rate of 0.4, we are able to get test accuracy of 93.6885%. So clearly, our model has improved from its original 89.41% accuracy.